In [1]:
import ee
import eemont  # noqa: F401
import geemap
import geemap.colormaps as cm
# import matplotlib.dates as mdates
# import matplotlib.pyplot as plt
# import pandas as pd
# import geopandas as gpd
# import seaborn as sns

from bisonlab.data import s2_sr_harmonized
from bisonlab.io import kml_to_geodataframe
from pathlib import Path

In [2]:
# ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AWgavdcYJ8M-In4if7nZu4RXyl9u10qMSlx6-7Oei5N96ZWprKVy5BJB4C0



Successfully saved authorization token.


### Load Parcels

In [3]:
filepath = Path().cwd().parent.parent / "data" / "local" / "WRTBI-Morton Soil Types.kml"

df = kml_to_geodataframe(filepath)

df = df.rename(columns={"Name": "subsection", "layer": "parcel"})
df = df.drop(columns='Description')

In [4]:
# Union all "mask" polygons and create a separate mask dataframe
idx = df["parcel"] == "mask"
df_mask = df[idx].dissolve(by="parcel").reset_index()

# Drop "mask" from parcel dataframe
df = df.drop(index=idx.index[idx])

In [5]:
# Convert parcels dataframe to ee.featureCollection
parcels = geemap.geopandas_to_ee(df)
mask = geemap.geopandas_to_ee(df_mask)

In [32]:
# m = geemap.Map()
# m.addLayer(parcels, {"color": "red"}, "Parcels")
# m.addLayer(mask, {"color": "blue"}, "mask")
# m.centerObject(parcels, 14)
# m

In [10]:
worldcover = ee.ImageCollection('ESA/WorldCover/v200').first()

In [7]:
start_date = "2017-03-28"
end_date = "2022-11-01"

In [8]:
s2 = s2_sr_harmonized(parcels, start_date, end_date).spectralIndices("NDVI")

In [9]:
img = s2.select('NDVI').closest("2022-08-08").first()

In [11]:
rap_veg = (
    ee.ImageCollection("projects/rangeland-analysis-platform/vegetation-cover-v3")
    .filterBounds(parcels)
)

In [ ]:
ImageCollection ‘projects/rangeland-analysis-platform/npp-partitioned-v3’

In [12]:
rap_bands = ['AFG', 'BGR', 'LTR', 'PFG', 'SHR', 'TRE']
rap_names = ['annual forb and grass', 'bare ground', 'litter', 'perennial forb and grass', 'shrub', 'tree']
rap_palette = ["006400", "ffbb22", "ffff4c", "f096ff", "fa0000", "b4b4b4"]
rap_band_names = dict(zip(rap_bands, rap_names))
rap_legend = dict(zip(rap_names, rap_palette))
rap_vis = {
    'min': 1,
    'max': 6,
    'palette': rap_palette
}

In [13]:
esri = ee.ImageCollection(
    "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m"
).mosaic()
esri_vis = {
    'min': 1,
    'max': 10,
    'palette': [
        "#1A5BAB",
        "#358221",
        "#A7D282",
        "#87D19E",
        "#FFDB5C",
        "#EECFA8",
        "#ED022A",
        "#EDE9E4",
        "#F2FAFF",
        "#C8C8C8",
    ],
}

In [14]:
rap = rap_veg.closest("2020").first() #.select('TRE')

In [16]:
a = rap.toArray().arrayArgmax().arrayGet(0).rename('label')

In [71]:
def reverse_mask(img, mask):
    """Mask out (remove) pixels from an image"""
    return img.mask(ee.Image(1).toByte().paint(mask, 0))

In [72]:
result = reverse_mask(img, mask)

In [73]:
m = geemap.Map()
m.add_basemap(basemap="SATELLITE")
m.addLayer(result, {"min": -1, "max": 1, "palette": cm.palettes.ndvi}, "NDVI")
m.centerObject(mask, 18)
m

Map(center=[43.1938352081597, -108.77802284064542], controls=(WidgetControl(options=['position', 'transparent_…

In [30]:
m = geemap.Map()
m.add_basemap(basemap="SATELLITE")

# m.addLayer(worldcover, {'bands': ['Map'],}, "ESA")
# m.addLayer(esri, esri_vis, "ESRI")

# m.add_legend(
#     title='ESA',
#     builtin_legend='ESA_WorldCover',
#     draggable=False,
#     position='bottomleft',
#     style={'bottom': '5px'},
# )
# m.add_legend(
#     title='ESRI',
#     builtin_legend='ESRI_LandCover',
#     draggable=False,
#     position='bottomright',
# )


# m.addLayer(rap.gt(0).selfMask(), {}, "")
# m.addLayer(a, {}, "")
m.addLayer(a, rap_vis, "rangelands")
m.add_legend(title='rangelands', legend_dict=rap_legend, draggable=False)

# m.addLayer(rap, {"min": 0, "max": 255, "palette": cm.palettes.viridis}, "rangelands")
# m.add_colorbar_branca(colors=rap_palette, vmin=1, vmax=6, categorical=True, step = 6, layer_name="rangeland")

m.addLayer(parcels.style(**{'color': 'red', 'fillColor': '00000000'}), {}, "Parcels")
m.addLayer(mask.style(**{'color': 'blue', 'fillColor': '00000000'}), {}, "mask")
m.addLayer(img, {"min": -1, "max": 1, "palette": cm.palettes.ndvi}, "NDVI")

m.centerObject(parcels, 13)
m

EEException: Image.mask, argument 'mask': Invalid type.
Expected type: Image<unknown bands>.
Actual type: FeatureCollection.

In [28]:
parcels.filter(
    ee.Filter.inList("parcel", ["Shoshone Tribe", "Hellyer Tribal Lease"])
)

In [106]:
s2.first()

In [ ]:
ts = s2.getTimeSeriesByRegions(
    reducer=ee.Reducer.mean(),
    collection=parcels,
    bands=None,
    scale=10,
    dateColumn="date",
    naValue=None,
)

In [ ]:
df = geemap.ee_to_pandas(ts)

In [ ]:
df = df.drop(
    columns=[
        "visibility",
        "descriptio",
        "tessellate",
        "extrude",
        "icon",
        "end",
        "begin",
        "timestamp",
        "altitudeMo",
    ]
)

In [ ]:
filename = "s2_sr_indices_mean.csv"
# df.to_csv(filename)
df = pd.read_csv(filename)

### Drop bad data

In [ ]:
# df[df['NDVI'] > 0.8]
# df[df['NDDI'] < -1.75e8]

In [ ]:
df = df.set_index("date")
df = df.drop("2021-11-08T18:22:16").reset_index()

### Plot

In [ ]:
a = df.melt(id_vars=["date", "Name"], value_vars=indices)

In [ ]:
g = sns.relplot(
    data=a,
    x="date",
    y="value",
    hue="Name",
    col="variable",
    col_wrap=1,
    kind="line",
    height=4,
    aspect=2,
    facet_kws=dict(sharey=False),
)
for ax in g.axes:
    ax.xaxis.set_major_locator(mdates.MonthLocator())
plt.gcf().autofmt_xdate()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(data=df, x="date", y="NDVI", hue="Name", ax=ax)
ax.xaxis.set_major_locator(mdates.MonthLocator())
fig.autofmt_xdate()